In [45]:
!pip install coincurve sha3 eth_utils ecpy numpy web3

In [46]:
import eth_utils
import ecpy
from ecpy import curves
import numpy as np
import web3

In [47]:
cv   = curves.Curve.get_curve('secp256k1')

In [48]:
print(cv)

{'name': 'secp256k1'
 'type': 'weierstrass'
 'size': 256
 'a': 0
 'b': 7
 'field': 115792089237316195423570985008687907853269984665640564039457584007908834671663
 'generator': <ecpy.curves.Point object at 0x7f32e0bcb640>
 'order': 115792089237316195423570985008687907852837564279074904382605163141518161494337
 'cofactor': 1}


In [49]:
print(cv.generator)

(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798 , 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)


In [50]:
order = cv.order
fieldSize = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F

In [51]:
fieldSize-cv.field

0

In [52]:
def numberToUint256(number):
    _ = hex(number).split('x')[1]
    return '0x'+'0'*(64-len(_))+_

In [53]:
def hashToCurve(pk, seed):
    #assumes the pubkey is a valid secp256k1 pt
    domsep = numberToUint256(1) #uint256 of 1 to account for the domain separator in chlink solidity contract
    h = int.from_bytes(eth_utils.keccak(hexstr = domsep+numberToUint256(pk.x)[2:]+numberToUint256(pk.y)[2:]+numberToUint256(seed)[2:]), 'big')
    while True:
        try:
            y2 = ((h*(h*h)%fieldSize)%fieldSize+7)%fieldSize
            #n % 4 = 3 => Legendre's formula for square root holds
            #x = sqrt(a) => x = +- a^((n+1)/4)
            #see Hardy, G. H.; Wright, E. M. (1980), An Introduction to the Theory of Numbers
            y = pow(y2, (fieldSize+1)//4, fieldSize)
            pt = curves.Point(h, y, cv)
            return pt if y % 2 == 0 else -pt
        except Exception as e:    
            print(str(e.value))
            #recursively hash
            h = int.from_bytes(eth_utils.keccak(hexstr = numberToUint256(h)), 'big')

In [54]:
#hashToCurve(publickey, 2000)

In [55]:
def ptToAddress(pt):
    return '0x'+eth_utils.keccak(pt.x.to_bytes(32, byteorder='big')+pt.y.to_bytes(32, byteorder='big'))[-20:].hex()

In [56]:
def marshalPoint(pt):
    print(pt.y.to_bytes(32, 'big'))
    return pt.x.to_bytes(32, 'big')+pt.y.to_bytes(32, 'big')

In [57]:
def ptToUint2562(pt):
    return [pt.x, pt.y]

In [58]:
def hashMuchToScalar(h, pubk, gamma, uw, v):
    chlinkDomSep = 2 #chlink domain separator
    return web3.Web3.solidity_keccak(
        ['uint256', 'uint256[2]', 'uint256[2]', 'uint256[2]', 'uint256[2]', 'address'],
        [chlinkDomSep, ptToUint2562(h), ptToUint2562(pubk), ptToUint2562(gamma), ptToUint2562(v), web3.Web3.to_checksum_address(uw)]
    )

In [59]:
def genProofWithNonce(seed, nonce, privkey):
    pkh = int.from_bytes(eth_utils.keccak(primitive=privkey), 'big')
    pubkey = cv.mul_point(pkh, cv.generator)
    h = hashToCurve(pubkey, seed)
    gamma = cv.mul_point(pkh, h)
    u = cv.mul_point(nonce, cv.generator)
    witness = ptToAddress(u)
    print(witness)
    v = cv.mul_point(nonce, h)
    print(v.x, v.y)
    c = int(hashMuchToScalar(
        h, pubkey, gamma, witness, v
    ).hex(), 16)
    s = (nonce - c*pkh)%cv.order
    outputHash = '0x'+eth_utils.keccak(hexstr = 
        numberToUint256(3)+marshalPoint(gamma).hex()
    ).hex() #gamma hash => vrf output, i.e., a uniform uint256 sample
    return {
        'pubkey': pubkey,
        'gamma': gamma,
        'c': c,
        's': s,
        'seed': seed,
        'output': outputHash
    }

In [60]:
genProofWithNonce(
    10, 20, 30
)

Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
0x811da72aca31e56f770fc33df0e45fd08720e157
74863086198399191924943238632167323619370939244480760622944205425442389914371 50881828270484193851872037685663306175837634488992815620786628956054752606330
b'Z\xcct\xb7\x9d\n\x8c.\xfe\xc6D\xd1`\xdf\x05q7\x00\x83\xc2\x1d[\xa7\xcf]\xd0t\xd3\xa2mt\xa2'


{'pubkey': <ecpy.curves.Point at 0x7f32e0bf51c0>,
 'gamma': <ecpy.curves.Point at 0x7f32e0bcb9c0>,
 'c': 2550015722850679518787319154538425662282961935893234342835482673578597365984,
 's': 94469602435952086750827151435547716300421105075696334838625225880192978608295,
 'seed': 10,
 'output': '0x63ed53ead4b20037021ec088219370af52418fa51e4005c379223d39f2415830'}

In [61]:
def PROJECTIVE_MULTIPLICATION(x1,z1,x2,z2):
    return x1*x2, z1*z2
def PROJECTIVE_SUBTRACTION(x1,z1,x2,z2):
    p1 = z2*x1
    p2 = -x2*z1
    return (p1+p2)%fieldSize, (z1*z2)%fieldSize
#ABOVE FIELD OPS BELOW GROUP OPS

In [62]:
def PROJECTIVE_ECCADDITION(pt1, pt2): #ECCADD in homogeneous projective coords chlink uses in their ctr
    x1,y1 = pt1.x, pt1.y
    x2,y2 = pt2.x, pt2.y
    z1,z2 = 1,1
    lx, lz = y2-y1, x2-x1
    sx, dx = PROJECTIVE_MULTIPLICATION(lx, lz, lx, lz)
    sx, dx = PROJECTIVE_SUBTRACTION(sx, dx, x1, z1)
    sx, dx = PROJECTIVE_SUBTRACTION(sx, dx, x2, z2)
    sy, dy = PROJECTIVE_SUBTRACTION(x1, z1, sx, dx)
    sy, dy = PROJECTIVE_MULTIPLICATION(sy, dy, lx, lz)
    sy, dy = PROJECTIVE_SUBTRACTION(sy, dy, y1, z1)
    if dx!=dy:
        sx*=dy
        sy*=dx
        sz=dx*dy
    else:
        sz = dx
    return sx%fieldSize, sy%fieldSize, sz%fieldSize

In [63]:
def modinvPRIME(a, ord):
    return pow(a, ord-2, ord)

In [64]:
def solProofAsInChlink(seed, nonce, privkey): #need not really be separate, I just noticed too late there are additional fields in sol ctr for gas-saving precomputes
    proof = genProofWithNonce(seed, nonce, privkey)
    u = cv.add_point(cv.mul_point(proof['c'], proof['pubkey']), cv.mul_point(proof['s'], cv.generator))
    hash = hashToCurve(proof['pubkey'], proof['seed'])
    print(hash.x, hash.y)
    cgw = cv.mul_point(proof['c'], proof['gamma'])
    shw = cv.mul_point(proof['s'], hash)
    _, _, PROJDENOM = PROJECTIVE_ECCADDITION(cgw, shw)
    zinv = modinvPRIME(PROJDENOM, fieldSize)
    print(
        (zinv*PROJDENOM)%fieldSize
    )
    return {
        'proof': proof,
        'uw': ptToAddress(u),
        'cgw': cgw,
        'shw': shw,
        'zinv': zinv
    }

In [65]:
prf = genProofWithNonce(10, 20, 30)

Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
0x811da72aca31e56f770fc33df0e45fd08720e157
74863086198399191924943238632167323619370939244480760622944205425442389914371 50881828270484193851872037685663306175837634488992815620786628956054752606330
b'Z\xcct\xb7\x9d\n\x8c.\xfe\xc6D\xd1`\xdf\x05q7\x00\x83\xc2\x1d[\xa7\xcf]\xd0t\xd3\xa2mt\xa2'


In [66]:
solProofAsInChlink(10, 20, 30)

Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
0x811da72aca31e56f770fc33df0e45fd08720e157
74863086198399191924943238632167323619370939244480760622944205425442389914371 50881828270484193851872037685663306175837634488992815620786628956054752606330
b'Z\xcct\xb7\x9d\n\x8c.\xfe\xc6D\xd1`\xdf\x05q7\x00\x83\xc2\x1d[\xa7\xcf]\xd0t\xd3\xa2mt\xa2'
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
8780222884449708748774725748080760201120738600334046747564455082096312727749 38102243232494913210339528110086982590896373180670496230211626875175386220432
1


{'proof': {'pubkey': <ecpy.curves.Point at 0x7f32e021f3c0>,
  'gamma': <ecpy.curves.Point at 0x7f32e0bea6c0>,
  'c': 2550015722850679518787319154538425662282961935893234342835482673578597365984,
  's': 94469602435952086750827151435547716300421105075696334838625225880192978608295,
  'seed': 10,
  'output': '0x63ed53ead4b20037021ec088219370af52418fa51e4005c379223d39f2415830'},
 'uw': '0x811da72aca31e56f770fc33df0e45fd08720e157',
 'cgw': <ecpy.curves.Point at 0x7f32f80d28c0>,
 'shw': <ecpy.curves.Point at 0x7f32e0bea680>,
 'zinv': 100194762916240629748119590467593311691077780522176707797321225309109196973747}

In [67]:
prov = web3.Web3(web3.HTTPProvider('https://sepolia.gateway.tenderly.co'))

In [68]:
prov.is_connected()

False

In [69]:
abi = '[{"inputs":[{"components":[{"internalType":"uint256[2]","name":"pk","type":"uint256[2]"},{"internalType":"uint256[2]","name":"gamma","type":"uint256[2]"},{"internalType":"uint256","name":"c","type":"uint256"},{"internalType":"uint256","name":"s","type":"uint256"},{"internalType":"uint256","name":"seed","type":"uint256"},{"internalType":"address","name":"uWitness","type":"address"},{"internalType":"uint256[2]","name":"cGammaWitness","type":"uint256[2]"},{"internalType":"uint256[2]","name":"sHashWitness","type":"uint256[2]"},{"internalType":"uint256","name":"zInv","type":"uint256"}],"internalType":"struct VRF.Proof","name":"proof","type":"tuple"},{"internalType":"uint256","name":"seed","type":"uint256"}],"name":"randomValueFromVRFProof","outputs":[{"internalType":"uint256","name":"output","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256[2]","name":"pk","type":"uint256[2]"},{"internalType":"uint256[2]","name":"gamma","type":"uint256[2]"},{"internalType":"uint256","name":"c","type":"uint256"},{"internalType":"uint256","name":"s","type":"uint256"},{"internalType":"uint256","name":"seed","type":"uint256"},{"internalType":"address","name":"uWitness","type":"address"},{"internalType":"uint256[2]","name":"cGammaWitness","type":"uint256[2]"},{"internalType":"uint256[2]","name":"sHashWitness","type":"uint256[2]"},{"internalType":"uint256","name":"zInv","type":"uint256"}],"name":"verifyVRFProof","outputs":[],"stateMutability":"view","type":"function"}]'

In [70]:
vrfctr = prov.eth.contract('0xEE52fbf97738Ae76d89f260b193f5b00d05D7401', abi = abi)

In [71]:
hex(solProofAsInChlink(10, 20, 30)['shw'].x)

Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
0x811da72aca31e56f770fc33df0e45fd08720e157
74863086198399191924943238632167323619370939244480760622944205425442389914371 50881828270484193851872037685663306175837634488992815620786628956054752606330
b'Z\xcct\xb7\x9d\n\x8c.\xfe\xc6D\xd1`\xdf\x05q7\x00\x83\xc2\x1d[\xa7\xcf]\xd0t\xd3\xa2mt\xa2'
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
8780222884449708748774725748080760201120738600334046747564455082096312727749 38102243232494913210339528110086982590896373180670496230211626875175386220432
1


'0x28d10d961afeebc98951429c55aba849ab00450de7c512a87cdc16b9fe685b67'

In [72]:
def ptToArr(pt):
    return [numberToUint256(pt.x), numberToUint256(pt.y)]
def ptToArrNat(pt):
    return [pt.x, pt.y]

In [73]:
def formatProofAsProof(proof):
    return [
        ptToArrNat(proof['proof']['pubkey']),
        ptToArrNat(proof['proof']['gamma']),
        proof['proof']['c'],
        proof['proof']['s'],
        proof['proof']['seed'],
        web3.Web3.to_checksum_address(proof['uw']),
        ptToArrNat(proof['cgw']),
        ptToArrNat(proof['shw']),
        proof['zinv']
    ]

In [74]:
_ = formatProofAsProof(solProofAsInChlink(np.random.randint(0, 1e6), np.random.randint(0, 1e6), np.random.randint(0, 1e6)))

0x1790a17ab86f60534c1b9a6c12ea7a7e2a49cc0f
91184012481621589291931270753460610378436359872908219737366472838428821182621 91363583620087044511614325052274571081484459280161846280009133211421300385522
b'\xa1^\xaf\x98g\xf8o\x84\xc42\xd0\xfc\x1fC\x91Rd3\x01\x9d]m:\x04\xf8\xe6k/\x96\xe0}\x89'
109360925089048043473142323561402532221447906968487931107011630325448725195854 3130026131399587132073243728663027936714315915252734286673438584224924004892
1


In [75]:
_ = formatProofAsProof(solProofAsInChlink(10, 20, 30))

Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
0x811da72aca31e56f770fc33df0e45fd08720e157
74863086198399191924943238632167323619370939244480760622944205425442389914371 50881828270484193851872037685663306175837634488992815620786628956054752606330
b'Z\xcct\xb7\x9d\n\x8c.\xfe\xc6D\xd1`\xdf\x05q7\x00\x83\xc2\x1d[\xa7\xcf]\xd0t\xd3\xa2mt\xa2'
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
Point not on curve
8780222884449708748774725748080760201120738600334046747564455082096312727749 38102243232494913210339528110086982590896373180670496230211626875175386220432
1


In [76]:
_

[[63155022883789688493371680209862490445514369463411226221659643962468555807992,
  99955949732387228775277083364683469913777587008053067871931724674521767595062],
 [91213803075109400468877361755141875464865437359194626313731230918609918698133,
  41069398726493601970477301634341161029642850897274024511218240778521136690338],
 2550015722850679518787319154538425662282961935893234342835482673578597365984,
 94469602435952086750827151435547716300421105075696334838625225880192978608295,
 10,
 '0x811da72aCA31e56F770Fc33DF0e45fD08720E157',
 [112015555460945632335719214645740881395415012627608701823581101668272029586953,
  96903784839656166123495876242624205453242699169996442179751859082329650468406],
 [18461878749406834228018327956670197390414480213258199266237144391360874830695,
  57008030688626927280283343239975755514347666948580257687850273130131065105656],
 100194762916240629748119590467593311691077780522176707797321225309109196973747]

In [77]:
vrfctr.functions.verifyVRFProof(*_).call() #verifying the proof onchain

[]

In [78]:
_ = formatProofAsProof(solProofAsInChlink(10, 20, 30))
vrfctr.functions.verifyVRFProof(*_)._encode_transaction_data()

'0x3848d2098ba079e946090d2313b0fcd608ac036918f4401f70aa35bd85b157017b7fdcf8dcfd100b73a4da5a9fba3c9d35be527c38d7e3ab87d727ab94257352b5c7cc36c9a92ed9f01842c4143b152750a9aadc5862df8604b9c900174c8d0a026722955acc74b79d0a8c2efec644d160df0571370083c21d5ba7cf5dd074d3a26d74a205a341fc0914ce271e528a3141325d9b84c28757628334eeab1fe2c691af38e0d0dbe6701593f475e594361bfe7f1c6de272174ea36d1b8427b9f0d2b12d60a7000000000000000000000000000000000000000000000000000000000000000a000000000000000000000000811da72aca31e56f770fc33df0e45fd08720e157f7a68ec0e252bf5ad3bf7a260b8be02404433c75ebc192f9ca08a6966e945609d63d993b857c88cabbf8e61496bb9326c623007baa48d7eb424ff20755dd963628d10d961afeebc98951429c55aba849ab00450de7c512a87cdc16b9fe685b677e0966e4b035eb261086b2422af3b6583070727aecdd97b9e8e2889fb1ff6cf8dd8439e4c18127f3970b7105142fbc290157094fd856b83a8b011d4d89dc66b3'